## Importing Helper Functions 

In [3]:
from helper_functions import *

In [4]:
df = get_clean_df()

William Friedkin
David Cronenberg
Allison Anders
Barry Levinson
Rodney Bennett
Jay Russell
Jake Kasdan
Ray Lawrence
Taylor Hackford
Frank Marshall
nan
nan
nan
William Friedkin
Peter Baldwin
George Hickenlooper
nan
Rick Rosenthal
Carl Erik Rinsch
Jim Jarmusch
nan
nan
Martyn Burke
Keith Gordon
John Woo
Michael Polish
Roy Ward Baker
David Arquette
John Sayles
Ken Loach
nan
William Wellman
Pauly Shore
Steve Boyum
Peter Cattaneo
Edward Dmytryk
Andy Sidaris
Ivan Nagy
Terence Young
Jon Turteltaub
nan
Paolo Sorrentino
Henning Schellerup
Allen Hughes|Albert Hughes
nan
Woody Allen
Yimou Zhang
Bill Froehlich
James Wong
nan
Pat Proft
Kevin Lima
Niels Arden Oplev
Matt Bettinelli-Olpin|Tyler Gillett
Anatole Litvak
Ira Miller
Steven Spielberg
Alan Alda
Otto Preminger
Terry Jones
Patrick Lussier
Tom Hanks
James Gray
Werner Herzog
Richard Kelly
Roy Rowland
nan
Ted Tetzlaff
William Beaudine
Bruce Beresford
nan
nan
Herbert Wilcox
John Gilling
Ernst Lubitsch
Cheryl Dunye
Ben Younger
Richard Linklater
Alan

In [ ]:
df

# Goal- Group by Genres

group by genres to look at distribution of domestic gross. Filtering by the last 10 years to get recent data. Narrowing time in theatres to days in order to standardize

In [ ]:
df

In [ ]:
df[["clean_domestic_gross", "domestic_gross_movie_gross", "domestic_gross_movie_budgets"]]

In [ ]:
df["domestic_gross"] = df["domestic_gross_movie_budgets"].replace('[$,()]', '', regex=True).astype(float)
df["domestic_gross"]

In [ ]:
#makes days in theatre column
df['theater_date'] = pd.to_datetime(df['theater_date'])
df['dvd_date'] = pd.to_datetime(df['dvd_date'])
df['days_in_theater'] = df['dvd_date'] - df['theater_date']
df['days_in_theater'] = df['days_in_theater'].dt.days #dt.dats converts timedelta to float
df['box_office_perday'] = df['domestic_gross'] / df['days_in_theater']
#gets what year the movie was released
df['release_year'] = df['theater_date'].dt.year

In [ ]:
df

## Filtering by start year to get the last 10 years only

In [ ]:
#only movies from 2012 to now
df_current= df[df["start_year"]>2012].drop_duplicates(subset=["movie_id"])
df_current

## We need this data to be normal so that we can run statistical tests on it in order to show relationships


### Cannot proceed with grouping by genre until the gross income data is normal

#### Creating graphs to eye-ball the distribution using seaborn

##### Bar chart

In [ ]:
#testing the distribution 
import seaborn as sns
sns.displot(df_current['domestic_gross'])

#### very skewed distribution

most of the movies made made less than $1,000,000. Only a few made over 2mil. This makes sense because only a few can be block busters.

In [ ]:
df_current.info()

##### Histogram

In [ ]:
sns.histplot(df_current['domestic_gross'])

Checked to see if making it a histogram would show any changes in the distribution. It did not.

##### Box Plot

In [ ]:
sns.boxplot(df_current['domestic_gross'])

Shows the same results are the previous data.

In [ ]:
df_current["domestic_gross"].mean()

The mean of the df is not reflective of what most movies made due to the high gross income the block-busters made.

In [ ]:
#Filtering for movies that made less than $1.5 mil ---> hoping this will be more normalized
df_filtered_gross= df[df["domestic_gross"]<150000000]
df_filtered_gross

In [ ]:
sns.histplot(df_filtered_gross['domestic_gross'])

Still very skewed. Probably cannot use this data for statistic tests due to it not being normal.

In [ ]:
df_filtered_gross["domestic_gross"].mean()
#mean= 29,132,181

# Conclusion 

We msot likely not going to be able to use gross domestic income in the movies that have come out in the last 10 years grouped by genres. The data was too skewed to be statistically tested. 

Idea: look into why the outlifers(movies with high domestic gross income) have done so well.

# Movies with domestic gross income> $4,000,000

Looking into why certain movie have made so much money- specific reasons?

In [ ]:
#Filtering for movies that made more than $4 mil ---> hoping this will be more normalized
df_filtered_gross_2= df[df["domestic_gross"]>4000000]
df_filtered_gross_2